In [1]:
import main
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/Users/jigs/anaconda3/lib/python3.11/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Load the file
df = pd.read_csv("Training_data_uhi_index 2025-02-04.csv")

# Preview
print(df.shape)
print(df.columns)
print(df.head())
print(df.isnull().sum())


(11269, 4)
Index(['Longitude', 'Latitude', 'datetime', 'UHI Index'], dtype='object')
   Longitude   Latitude          datetime  UHI Index
0 -73.919037  40.814292  24-07-2021 15:53   1.034616
1 -73.918978  40.814365  24-07-2021 15:53   1.028125
2 -73.918927  40.814433  24-07-2021 15:53   1.028125
3 -73.918875  40.814500  24-07-2021 15:53   1.025961
4 -73.918827  40.814560  24-07-2021 15:53   1.025961
Longitude    0
Latitude     0
datetime     0
UHI Index    0
dtype: int64


In [3]:
df.rename(columns={
    'Longitude': 'longitude',
    'Latitude': 'latitude',
    'datetime': 'timestamp',
    'UHI Index': 'uhi_index'
}, inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'], format="%d-%m-%Y %H:%M")

In [4]:
# # Run the full batch extraction and save the result
# features_df = main.batch_extract_features_with_cache(df, chunk_size=500)
#
# # Optionally, combine with the original UHI index
# final_df = pd.concat([df.reset_index(drop=True), features_df], axis=1)
#
# # Save for training
# final_df.to_csv("uhi_with_features.csv", index=False)


In [5]:
# # Load the dataset
# df_features = pd.read_csv("uhi_with_features.csv")
#
# # Basic shape and structure
# print(f"Shape: {df_features.shape}")
# print("First few columns:", df_features.columns[:10].tolist())
# print("Last few columns:", df_features.columns[-10:].tolist())
# print("\nData types:\n", df_features.dtypes.head())
# print("\nMissing values:\n", df_features.isnull().sum().head())
# print("\nUHI Index stats:\n", df_features["uhi_index"].describe())
#
# # Check for NaNs
# print("\nAny missing values in full dataset?:", df_features.isnull().values.any())
#
# # Plot UHI Index distribution
# plt.figure(figsize=(8, 4))
# sns.histplot(df_features["uhi_index"], bins=50, kde=True)
# plt.title("Distribution of UHI Index")
# plt.xlabel("UHI Index")
# plt.ylabel("Frequency")
# plt.tight_layout()
# plt.show()
#
# # Correlation with UHI index (just for first 20 features)
# feature_cols = df_features.columns.difference(["latitude", "longitude", "timestamp", "uhi_index"])
# correlations = df_features[feature_cols].iloc[:, :20].corrwith(df["uhi_index"])
# print("\nSample correlations with UHI Index:\n", correlations.sort_values(ascending=False).head())
#


In [6]:
# # Load one cached feature file manually
# chunk = pd.read_csv("features_cache/chunk_0.csv")  # Adjust path if needed
#
# print("Chunk shape:", chunk.shape)
# print(chunk.head())
# print(chunk.isnull().sum().head())


In [7]:
# from mosaiks import get_features
#
# latitudes = [40.814292]
# longitudes = [-73.919037]
# datetime = "2021-07-24"
#
# features_df = get_features(
#     latitudes=latitudes,
#     longitudes=longitudes,
#     datetime=datetime,
#     satellite_name="sentinel-2-l2a",
#     image_resolution=10,
#     image_bands=["B02", "B03", "B04", "B08"],
#     image_width=3000,
#     n_mosaiks_features=4000,
#     parallelize=False,
#     model_device="cpu"
# )
#
# print(features_df.head())


In [8]:
# # Check if the entire DataFrame is full of NaNs
# if features_df.isnull().all().all():
#     print("❌ features_df is completely filled with NaNs.")
# else:
#     print("✅ features_df contains valid (non-NaN) values.")


In [9]:
sample_df = df[['latitude', 'longitude']].head(10)

features = main.batch_extract_features_with_cache(
    sample_df,
    chunk_size=10,
    cache_dir="features_cache_test",
    datetime="2021-07-17/2021-07-31",  # broad enough
    satellite="sentinel-2-l2a",
    bands=["B02", "B03", "B04", "B08"],
    resolution=10
)


[Fetch] Processing chunk 0
[Debug] Locations sample: [(40.81429167, -73.91903667), (40.814365, -73.91897833), (40.81443333, -73.91892667)]
[Step 1] Converting locations to GeoDataFrame
[Step 2] Connecting to Planetary Computer STAC API
[Step 3] Running STAC search with fixed NYC bounding box
[Debug] Bounding box used: [-74.03, 40.69, -73.9, 40.88]


/Users/jigs/anaconda3/lib/python3.11/site-packages/pystac_client/item_search.py:826: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


[Step 3a] Found 1 matching items
[Step 4] Signing STAC items
[Debug] First item asset keys: ['AOT', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'B8A', 'SCL', 'WVP', 'visual', 'preview', 'safe-manifest', 'granule-metadata', 'inspire-metadata', 'product-metadata', 'datastrip-metadata', 'tilejson', 'rendered_preview']
[Step 5] Stacking assets using stackstac
[Debug] Stacked dimensions: ('time', 'band', 'y', 'x')
[Debug] Stacked shape: (1, 4, 11118, 10165)
[Debug] time size: 1
[Debug] band size: 4
[Error] Attempt 1 for chunk 0 failed: Index contains duplicate entries, cannot reshape
[Debug] Locations sample: [(40.81429167, -73.91903667), (40.814365, -73.91897833), (40.81443333, -73.91892667)]
[Step 1] Converting locations to GeoDataFrame
[Step 2] Connecting to Planetary Computer STAC API
[Step 3] Running STAC search with fixed NYC bounding box
[Debug] Bounding box used: [-74.03, 40.69, -73.9, 40.88]


/Users/jigs/anaconda3/lib/python3.11/site-packages/pystac_client/item_search.py:826: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


[Step 3a] Found 1 matching items
[Step 4] Signing STAC items
[Debug] First item asset keys: ['AOT', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'B8A', 'SCL', 'WVP', 'visual', 'preview', 'safe-manifest', 'granule-metadata', 'inspire-metadata', 'product-metadata', 'datastrip-metadata', 'tilejson', 'rendered_preview']
[Step 5] Stacking assets using stackstac
[Debug] Stacked dimensions: ('time', 'band', 'y', 'x')
[Debug] Stacked shape: (1, 4, 11118, 10165)
[Debug] time size: 1
[Debug] band size: 4
[Error] Attempt 2 for chunk 0 failed: Index contains duplicate entries, cannot reshape
[Debug] Locations sample: [(40.81429167, -73.91903667), (40.814365, -73.91897833), (40.81443333, -73.91892667)]
[Step 1] Converting locations to GeoDataFrame
[Step 2] Connecting to Planetary Computer STAC API
[Step 3] Running STAC search with fixed NYC bounding box
[Debug] Bounding box used: [-74.03, 40.69, -73.9, 40.88]


/Users/jigs/anaconda3/lib/python3.11/site-packages/pystac_client/item_search.py:826: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


[Step 3a] Found 1 matching items
[Step 4] Signing STAC items
[Debug] First item asset keys: ['AOT', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'B8A', 'SCL', 'WVP', 'visual', 'preview', 'safe-manifest', 'granule-metadata', 'inspire-metadata', 'product-metadata', 'datastrip-metadata', 'tilejson', 'rendered_preview']
[Step 5] Stacking assets using stackstac
[Debug] Stacked dimensions: ('time', 'band', 'y', 'x')
[Debug] Stacked shape: (1, 4, 11118, 10165)
[Debug] time size: 1
[Debug] band size: 4
[Error] Attempt 3 for chunk 0 failed: Index contains duplicate entries, cannot reshape
[Fail] Skipping chunk 0


In [10]:
df_chunk = pd.read_csv("features_cache_test/chunk_0.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'features_cache_test/chunk_0.csv'

In [ ]:
df_chunk

In [ ]:
df_chunk.shape


In [ ]:
df_chunk.isnull().sum()